In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from bioservices import UniProt

In [2]:
service = UniProt()
#constructing query
query = "cluster:(uniprot:Q6CZT4 identity:0.9)"
# Querying the database
result1= service.search(query, frmt="fasta")
result2= service.search(query,database="uniparc",frmt="fasta")
peclyase3uniprot = result1 + result2
#Saving to file
savefile = open("peclyase3uniprot.txt","w")
savefile.write(peclyase3uniprot)
savefile.close()

In [3]:
seqdata = "".join(["MKYLLPSTAAGLLLLAAQPTMAANTGGYATTDGGNVAGAVNKTARSMQDIIDIIEEAKLD",
"SKGKKVKGGAYPLIITYNGNEDALIKAAENNICGQWSKDARGVEIKEFTKGVTIIGTNGS",
"SANFGIWLTKSSDVIIRNMRFGYMPGGAQDGDAIRIDNTPNVWIDHNEIFAKNFECQGTK",
"DGDTTFESAIDIKKASTNVTVSYNYIHGIKKVGLSGFSSSDTGRDLTYHHNIYDDVNARL",
"PLQRGGQVHAYNNLYTGITSSGLNVRQKGIALIERNWFENAKNPVTSRYDGSNFGTWELR",
"NNNIMSPADFAKYNITWDKDTKAYVNAEDWKSTGTFASVPYSYSPVSPQCVKDKLANYAG"
"VNKNLAVLTAANCN"])
refseq = SeqRecord(seq=Seq(seqdata),
id="sp|Q6CZT2|PLY3_PECAS",
description="Pectate lyase 3 OS=Pectobacterium atrosepticum (strain SCRI 1043 / ATCC BAA-672) OX=218491 GN=pel3 PE=3 SV=1")

In [4]:
blastresult = NCBIWWW.qblast("blastp", "nr", refseq.format("fasta"), entrez_query="pectobacterium[ORGN]")

In [5]:
result= NCBIXML.read(blastresult)

In [6]:
blastaccessions = []
for alignment in result.alignments:
    for hsp in alignment.hsps:
        if hsp.align_length / result.query_length == 1 and hsp.identities/ hsp.align_length >= 0.90:
            blastaccessions.append(alignment.accession)
print(blastaccessions)

['WP_011095543', 'WP_048262716', 'WP_129708440', 'WP_039324738', 'WP_039304762', 'WP_107168870', 'WP_137742183', 'WP_014701648', 'WP_039529856', 'WP_010297068', 'WP_039498079', 'WP_039516338', 'WP_014916842', 'WP_095700775', 'WP_094370557', 'WP_095993019', 'WP_119873299', 'WP_039503864', 'WP_180781084', 'WP_103862631', 'WP_040032980', 'WP_039492146', 'WP_044205489', 'WP_039277268', 'WP_039362804', 'WP_039484584', 'WP_040033289', 'WP_110163332', 'WP_039557351', 'WP_102116850', 'WP_174870902', 'AAB82289', 'WP_010277583', 'MBA5602669', 'MBA5201940', 'WP_180779231', 'WP_180743239', 'WP_015841959', 'CAA57441', 'WP_119889190', 'WP_109412366', 'WP_138254708', 'WP_127157730', 'WP_005969502', 'WP_039353884']


In [8]:
from Bio import Entrez
Entrez.email = "jakesgames212@hotmail.co.uk"
with open("peclyase3blast.fasta","w") as outfh:
    for accessions in blastaccessions:
        output = Entrez.efetch(db= "protein",id=accessions, rettype="fasta", retmode="text").read()
        outfh.write(output)

In [9]:
#parsing the uniprotKB sequences with SeqIO
uniprot_seqs = [_ for _ in SeqIO.parse("peclyase3uniprot.txt", "fasta")]
#isolating the sequence ids
uniprot_ids = [_.id for _ in uniprot_seqs]

#parsing the Blastp sequences with SeqIO
blast_seqs = [_ for _ in SeqIO.parse("peclyase3blast.fasta", "fasta")]
#isolating the sequence ids
blast_ids = [_.id for _ in blast_seqs]

In [10]:
#For each blast sequence, search uniprot using the crossreference refseq
# Convert this to tabular format and isolate only the sequences
# For each blast sequence searched, if there was no uniprot result...print the blast sequence
new_blast_seqs = []
for seq in blast_seqs:
    result = service.search("database:(type:refseq %s)" % seq.id, frmt="tab", columns="id").split("\n")[1:-1]
    if len(result) == 0:
        new_blast_seqs.append(seq)

In [11]:
seq_combined = uniprot_seqs + new_blast_seqs

len(seq_combined)

SeqIO.write(seq_combined, "combined.fasta", "fasta")

76